## Task1 匹配手机号码

号码规则如下
- 以 0086 或 +86 开头，也可以没有
- 若有开头，开头与11位数字间可以有也可以没有分隔符(仅考虑' '或'-')
- 11位数字内部：
	- 第一位数字为1，第二位数字（3-9），之后的数字不限制
	- 数字可以 3-4-4 的组合分隔，分隔符仍仅考虑' '或'-'
    
例如：13812345678，+86-15912345678， 178-1234-5678， 0086-189 1234 5678

In [101]:
import re

# 完成正则
# TODO
regex = re.compile(r"(00)?(\+)?(86)?[-| ]?1[3-9]\d[-| ]?(\d+)[-| ]?(\d+)")


print(regex.match("13812345678"))
print(regex.match("+86-15912345678"))
print(regex.match("178-1234-5678"))
print(regex.match("0086-189 1234 5678"))

<re.Match object; span=(0, 11), match='13812345678'>
<re.Match object; span=(0, 15), match='+86-15912345678'>
<re.Match object; span=(0, 13), match='178-1234-5678'>
<re.Match object; span=(0, 18), match='0086-189 1234 5678'>


## Task 2 解析京东商品页

使用到的内置模块：
    re  urllib

使用urllib获取网页，网页保存到本地

并使用re解析出商品信息，仅需要解析出该页即可，不需要爬取下一页


需要解析的字段为

- data-sku
- title
- href
- p-price (￥ 以及 价格)

单个商品的html代码如下一栏所示


结果：
[商品sku， 商品名, 币种， 价格]  
示例：
['7512626', '联想(Lenovo)拯救者Y7000P英特尔酷睿 i7 15.6英寸高色域游戏笔记本电脑(i7-8750H 8G 512G GTX1060 144Hz)', '￥', '8599.00']


提示：   
\s\S 或者 \w\W 可以匹配任意字符, [\s\S]+? 其中的问号表示非贪婪匹配  

**截取的单个商品html示例**

```html
<li data-sku="8674557" class="gl-item">
	<div class="gl-i-wrap">
					<div class="p-img">
						<a target="_blank" title="A面金属抗压防划抗腐蚀，144Hz更高刷新率消除图像闪烁抖动画面更流畅，三边窄边框，512G_PCIE大固态硬盘存储更快速【玩转开学季】" href="//item.jd.com/8674557.html" onclick="searchlog(1,8674557,1,2,'','flagsClk=1094718088')">
							<img width="220" height="220" class="err-product" data-img="1" source-data-lazy-img="//img12.360buyimg.com/n7/jfs/t1/27368/40/7173/165300/5c67ed1eEc7ad4926/fd6035633ebff93a.jpg" />
</a>						<div data-lease="" data-catid="1105" data-venid="1000000157" data-presale=""></div>
					</div>
					<div class="p-price">
<strong class="J_8674557" data-done="1"><em>￥</em><i>8599.00</i></strong>					</div>
					<div class="p-name p-name-type-2">
						<a target="_blank" title="A面金属抗压防划抗腐蚀，144Hz更高刷新率消除图像闪烁抖动画面更流畅，三边窄边框，512G_PCIE大固态硬盘存储更快速【玩转开学季】" href="//item.jd.com/8674557.html" onclick="searchlog(1,8674557,1,1,'','flagsClk=1094718088')">
							<em>联想(Lenovo)拯救者Y7000P英特尔酷睿 i7 15.6英寸高色域游戏<font class="skcolor_ljg">笔记本</font>电脑(i7-8750H 8G 512G GTX1060 144Hz)</em>
							<i class="promo-words" id="J_AD_8674557">A面金属抗压防划抗腐蚀，144Hz更高刷新率消除图像闪烁抖动画面更流畅，三边窄边框，512G_PCIE大固态硬盘存储更快速【玩转开学季】</i>
						</a>
					</div>
					<div class="p-commit">
						<strong><a id="J_comment_8674557" target="_blank" href="//item.jd.com/8674557.html#comment" onclick="searchlog(1,8674557,1,3,'','flagsClk=1094718088')"></a></strong>
					</div>
					<div class="p-shop" data-selfware="1" data-score="5" data-reputation="98">
<span class="J_im_icon"><a target="_blank" class="curr-shop" onclick="searchlog(1,1000000157,0,58)" href="//mall.jd.com/index-1000000157.html" title="联想电脑京东自营旗舰店">联想电脑京东自营旗舰店</a></span>					</div>
					<div class="p-icons" id="J_pro_8674557" data-done="1">
			<i class="goods-icons J-picon-tips J-picon-fix" data-idx="1" data-tips="京东自营，品质保障">自营</i>
						<i class="goods-icons J-picon-tips J-picon-fix" data-tips="只为品质生活" style="background:#aa09ed;">品质优选</i>
					</div>
					<div class="p-operate">
						<a class="p-o-btn contrast J_contrast" data-sku="8674557" href="javascript:;" onclick="searchlog(1,8674557,1,6,'','flagsClk=1094718088')"><i></i>对比</a>
						<a class="p-o-btn focus J_focus" data-sku="8674557" href="javascript:;" onclick="searchlog(1,8674557,1,5,'','flagsClk=1094718088')"><i></i>关注</a>
						<a class="p-o-btn addcart" href="//cart.jd.com/gate.action?pid=8674557&pcount=1&ptype=1" target="_blank" onclick="searchlog(1,8674557,1,4,'','flagsClk=1094718088')" data-limit="0"><i></i>加入购物车</a>
					</div>
	</div>
</li>
```

In [102]:
import re

from urllib.request import urlopen
from urllib.parse import quote
import pickle

def jd_crawler(keyword):
    # URL 只允许一部分 ASCII 字符（数字字母和部分符号），汉字不符合 URL 标准的，使用quote进行编码
    url = 'https://search.jd.com/Search?keyword='+ quote(keyword) + '&enc=utf-8'
    
    # 读取网页
    html = urlopen(url).read()
    
    # 保存该html网页至本地
    # TODO 1.
    f = open("localweb.html", 'wb')
    f.write(html)
    
    html_decoded = html.decode('utf-8')
    
    # 正则
    # TODO 2
    
    regex = r'data-sku="(\d*)"[\s\S]*?<em>(.)</em><i>(.*)</i>[\s\S]*?<em>(.*)</em>'
    
    
    m = re.finditer(regex, html_decoded)
    result = []
    for i, good in enumerate(m):
        result.append([good.group(1), good.group(4).replace('<font class="skcolor_ljg">', '').replace('</font>', ''),
                      good.group(2), good.group(3)])
        
    for res in result:
        print(res)
    
    

jd_crawler("笔记本")

['6076583', '联想ThinkPad 翼480（04CD）英特尔酷睿i5 14英寸轻薄窄边框笔记本电脑（i5-8250U 8G 256SSD ）冰原银', '￥', '4899.00']
['6076583', '联想(Lenovo)拯救者Y7000英特尔酷睿i5 15.6英寸游戏笔记本电脑( i5-8300H 8G 512G SSD GTX1050 黑)', '￥', '5498.00']
['100000769466', '荣耀MagicBook 14英寸轻薄窄边框笔记本电脑（AMD锐龙5 8G 256G FHD IPS 正版Office）冰河银', '￥', '3999.00']
['7512626', '联想(Lenovo)拯救者Y7000P英特尔酷睿 i7 15.6英寸高色域游戏笔记本电脑(i7-8750H 8G 512G GTX1060 144Hz)', '￥', '8599.00']
['8674557', '戴尔灵越14 燃 14英寸英特尔酷睿i5轻薄窄边框笔记本电脑(i5-8265U 8G 256G MX150 2G独显)银', '￥', '5199.00']
['100000117758', '联想ThinkPad 翼480 14英寸轻薄窄边框笔记本电脑（八代英特尔酷睿i5 8G 128GSSD+500G 2G独显 FHD）冰原银', '￥', '5499.00']
['6072622', '联想(Lenovo)330C 英特尔酷睿i5 15.6英寸商务影音笔记本电脑(i5-8250U 4G 1T+128G SSD 独显MX110)黑', '￥', '3999.00']
['7765111', 'Apple MacBook Air 13.3英寸笔记本电脑 银色(2017款Core i5 处理器/8GB内存/128GB闪存 MQD32CH/A)', '￥', '6488.00']
['5225346', '联想ThinkPad E485（01CD）14英寸笔记本电脑（锐龙5-2500U 8G双通道 256GSSD FHD Win10）黑色', '￥', '4399.00']
['100000060090', '戴尔DELL游匣G315.6英寸英特尔酷睿i5游戏笔记本电脑(i5-8300H 8G 128G 1T GTX1050Ti 4G独显

##  Task3 使用 python 实现 设计模式中的观察者模式

观察者模式也被称为发布-订阅（Publish/Subscribe）模式。当这个主题对象状态变化时，会通知所有观察者对象并作出相应处理逻辑。

在本问题中，可以形象地理解成，Publisher为公众号，Reader为订阅公众号的人。需要实现订阅，退订，Publisher发布新消息，需要推送给所有已订阅的人。


![观察者模式UML类图](./img/观察者模式UML类图.png)

**Publisher（发布者）**
    - 保存订阅的读者
    - 保存已发布的消息
    - 发布新消息时，发布给所有已订阅的读者
    
    共有三个发布者：PKUPublisher， THUPublisher， RUCPublisher
    

**Reader（读者）**
    - 订阅/取消订阅 发布者
    - 保存发布者发来的消息
    
    读者分为两种类型：
        - Type1：不区分不同发布者的消息
        - Type2：区分不同发布者的消息
        
    共有三个读者：alice， bob， tom  alice和bob为Type1， tom为Type2
    
    
提示：   
    Publisher： 
        - 保存读者应使用set，避免重复订阅
        - 需要实现读者订阅/取消订阅，向读者发布消息功能
        
        
   Reader：
        - 保存发布者应使用set，避免重复
        - 区分两种类型的读者的消息保存方式的subscribe/receive方法的不同
        
   




注：各Publisher和Reader要求实现 str()功能   
如：   
str(pkuPublisher) 格式化输出：   
Publisher Name: PKUPublisher   
Subscirbed Reader: ['tom', 'alice']   
Publisher News: ['思想自由，兼容并包']   

str(tom) 格式化输出：   
Reader Name: tom   
Subscirbed Publisher: ['THUPublisher', 'PKUPublisher', 'RUCPublisher']   
Reader News: {'PKUPublisher': ['思想自由，兼容并包'], 'THUPublisher': ['自强不息，厚德载物'], 'RUCPublisher': ['实事求是']}   


#### 参考资料
[1. Observer pattern Wiki](https://en.wikipedia.org/wiki/Observer_pattern)   
[2. 设计模式之观察者模式（c++）](https://www.cnblogs.com/carsonzhu/p/5770253.html)

In [103]:
# 发布者的基类
class BasePublisher(object):
    
    def __init__(self):
        # BasePublisher 的初始化方法
        # TODO
        self.reader = set()
        self.news = []

    
    def subscribeReader(self, reader):
        # Publisher 端的订阅函数
        # TODO
        self.reader.add(reader)
        
        
    
    def unsubscribeReader(self, reader):
        # Publisher 端的取消订阅函数
        # TODO
        self.reader.remove(reader)
        
        
    def notifyReader(self, new):
        pass
        
    def __str__(self):
        tmpread = []
        for i in self.reader:
            tmpread.append(i.name)
        return "Publisher Name: {}\nSubscribed Reader: {}\nPublisher News: {}".format(self.name, tmpread, self.news)
    
# 北大
class PKUPublisher(BasePublisher):
    
    def __init__(self):
        # PKUPublisher 的初始化方法
        # TODO
        BasePublisher.__init__(self)
        self.name = 'PKUPublisher'
        
    def notifyReader(self, new):
        # Publisher 向 Reader 发布消息
        # TODO
        self.news.append(new)
        for i in self.reader:
            i.receiveNew(self, new)
      
        
# 清华
class THUPublisher(BasePublisher):
    
    def __init__(self):
        # THUPublisher 的初始化方法
        # TODO
        BasePublisher.__init__(self)
        self.name = 'THUPublisher'
        
    def notifyReader(self, new):
        # Publisher 向 Reader 发布消息
        # TODO
        self.news.append(new)
        for i in self.reader:
            i.receiveNew(self, new)
        
            
# 人大
class RUCPublisher(BasePublisher):
    
    def __init__(self):
        # RUCPublisher 的初始化方法
        # TODO
        BasePublisher.__init__(self) 
        self.name = 'RUCPublisher'
        
    def notifyReader(self, new):
        # Publisher 向 Reader 发布消息
        # TODO
        self.news.append(new)
        for i in self.reader:
            i.receiveNew(self, new)    


# 读者基类
class BaseReader(object):
    
    def __init__(self):
        # BaseReader 的初始化方法
        # TODO
        self.publisher = set()    
    
    def subscribeToPublisher(self, publisher):
        # Reader向Publisher订阅
        # TODO
        publisher.subscribeReader(self)
          
    def unsubscribeToPublisher(self, publisher):
        # Reader向Publisher取消订阅
        # TODO
        publisher.unsubscribeReader(self)
           
    def receiveNew(self, publisher, new):
        pass
    
    
    

# 第一种读者
class ReaderType1(BaseReader):
    
    def __init__(self, _name):
        # TODO
        BaseReader.__init__(self)
        self.name = _name
        self.news = dict()
        
    def subscribeToPublisher(self, publisher):
        # Reader向Publisher订阅
        # TODO
        publisher.subscribeReader(self)
        self.publisher.add(publisher.name)
        
    def receiveNew(self, publisher, new):
        # Reader 从 Publisher 接收消息
        # TODO
        self.news[publisher.name] = [new]
        
    def __str__(self):
        listForPrint = set()
        for key, value in self.news.items():
            listForPrint.add(key+':'+str(value))
        return "Reader Name: {}\nSubscribed Publisher: {}\nReader News: {}".format(self.name, list(self.publisher), listForPrint)
        


# 第二种读者
class ReaderType2(BaseReader):
    
    def __init__(self, _name):
        # TODO
        BaseReader.__init__(self)
        self.name = _name
        self.news = []
        
    def subscribeToPublisher(self, publisher):
        # Reader向Publisher订阅
        # TODO
        publisher.subscribeReader(self)
        self.publisher.add(publisher.name)
        
    def receiveNew(self, publisher, new):
        # TODO
        self.news.append(new) 
        
    def __str__(self):
        return "Reader Name: {}\nSubscirbed Publisher: {}\nReader News: {}".format(self.name, list(self.publisher), self.news)         
        
        
# 创建三个发布者对象
pkuPublisher = PKUPublisher()
thuPublisher = THUPublisher()
rucPublisher = RUCPublisher()

# 创建三位读者，alice 和 bob 为 第一类， tom 为 第二类
alice = ReaderType1('alice')
bob = ReaderType1('bob')
tom = ReaderType2('tom')

# alice 订阅了 pku， thu
alice.subscribeToPublisher(pkuPublisher)
alice.subscribeToPublisher(thuPublisher)

# bob 订阅了 thu， ruc
bob.subscribeToPublisher(thuPublisher)
bob.subscribeToPublisher(rucPublisher)

# tom 订阅了 pku， thu， ruc
tom.subscribeToPublisher(pkuPublisher)
tom.subscribeToPublisher(thuPublisher)
tom.subscribeToPublisher(rucPublisher)


# 发布者发布新消息
pkuPublisher.notifyReader("思想自由，兼容并包")
thuPublisher.notifyReader("自强不息，厚德载物")
rucPublisher.notifyReader("实事求是")

print(str(pkuPublisher))
print(str(thuPublisher))
print(str(rucPublisher))
print(str(alice))
print(str(bob))
print(str(tom))


Publisher Name: PKUPublisher
Subscribed Reader: ['tom', 'alice']
Publisher News: ['思想自由，兼容并包']
Publisher Name: THUPublisher
Subscribed Reader: ['tom', 'bob', 'alice']
Publisher News: ['自强不息，厚德载物']
Publisher Name: RUCPublisher
Subscribed Reader: ['tom', 'bob']
Publisher News: ['实事求是']
Reader Name: alice
Subscribed Publisher: ['PKUPublisher', 'THUPublisher']
Reader News: {"PKUPublisher:['思想自由，兼容并包']", "THUPublisher:['自强不息，厚德载物']"}
Reader Name: bob
Subscribed Publisher: ['RUCPublisher', 'THUPublisher']
Reader News: {"THUPublisher:['自强不息，厚德载物']", "RUCPublisher:['实事求是']"}
Reader Name: tom
Subscirbed Publisher: ['PKUPublisher', 'RUCPublisher', 'THUPublisher']
Reader News: ['思想自由，兼容并包', '自强不息，厚德载物', '实事求是']
